In [353]:
import math
import random
import statistics
import scipy.stats

In [305]:
DAYS = 100
HIRE = 1
WORKDAYS = 3 * DAYS
DIST = scipy.stats.gamma(a=1)

In [386]:
def decide(day, candidate, _):
    if day + 1 == DAYS:
        return True
    days = DAYS - day
    workdays = days * candidate
    return workdays >= WORKDAYS

def decide2a(day, candidate, _):
    if day + 1 == DAYS:
        return True
    days = DAYS - day
    workdays = days * candidate
    p_next_will_be_worse = 1.0
    for next_day in range(day + 1, DAYS):
        rem_days = DAYS - next_day
        p_next_will_be_worse *= DIST.cdf(workdays / rem_days)
    return p_next_will_be_worse > 0.5

def decide2b(day, candidate, _):
    if day + 1 == DAYS:
        return True
    days = DAYS - day
    workdays = days * candidate
    p_next_will_be_worse = 1.0
    for next_day in range(day + 1, DAYS):
        rem_days = DAYS - next_day
        p_next_will_be_worse *= DIST.cdf(workdays / rem_days)
    return p_next_will_be_worse > 0.1

def decide2c(day, candidate, _):
    if day + 1 == DAYS:
        return True
    days = DAYS - day
    workdays = days * candidate
    p_next_will_be_worse = 1.0
    for next_day in range(day + 1, DAYS):
        rem_days = DAYS - next_day
        p_next_will_be_worse *= DIST.cdf(workdays / rem_days)
    return p_next_will_be_worse > 0.01

def decide3(day, candidate, _):
    if day + 1 == DAYS:
        return True
    return candidate >= DIST.mean()

def decide4(day, candidate, past_candidates):
    if day + 1 == DAYS:
        return True
    if day < DAYS / math.e:
        return False
    return candidate >= max(past_candidates)

ALGORITHMS = [decide, decide2a, decide2b, decide2c, decide3, decide4]

In [387]:
def simulate(algorithm):
    candidates = list(DIST.rvs(size=DAYS))
    hire_days = []
    workdays = 0
    for day, candidate in enumerate(candidates):
        d = algorithm(day, candidate, candidates[:day])
        if d:
            days = DAYS - day
            workdays += days * candidate
            hire_days.append(day)
            if len(hire_days) == HIRE:
                break
    return workdays, hire_days

In [389]:
SIMULATIONS = 1000

results = []
for algorithm in ALGORITHMS:
    simulations = []
    for _ in range(SIMULATIONS):
        simulations.append(simulate(algorithm))
    results.append((algorithm, simulations))

for alg, simulations in results:
    success_rate = sum(1 for workdays, _ in simulations if workdays >= WORKDAYS) / SIMULATIONS
    median = statistics.median(workdays for workdays, _ in simulations)
    worst = min(workdays for workdays, _ in simulations)
    best = max(workdays for workdays, _ in simulations)
    typical_first_hire_day = statistics.median(hire_days[0] for _, hire_days in simulations)
    print(f"{alg.__name__=} {success_rate=:%} {median=} {best=} {worst=} {typical_first_hire_day=}")

alg.__name__='decide' success_rate=68.600000% median=325.87313503150233 best=1069.7996931627836 worst=0.0021864579039915584 typical_first_hire_day=17.0
alg.__name__='decide2a' success_rate=59.600000% median=323.4917111460014 best=1033.887655362897 worst=10.0149141510897 typical_first_hire_day=17.0
alg.__name__='decide2b' success_rate=38.100000% median=276.44276716080765 best=859.2593368458736 worst=105.41612474096588 typical_first_hire_day=7.0
alg.__name__='decide2c' success_rate=25.200000% median=236.14194209088225 best=957.1551839752563 worst=131.18500167871753 typical_first_hire_day=4.0
alg.__name__='decide3' success_rate=13.100000% median=163.56902362674833 best=678.7502536990967 worst=92.22246207966637 typical_first_hire_day=1.0
alg.__name__='decide4' success_rate=6.000000% median=119.52407557112554 best=505.9621128024317 worst=0.0037996832457468247 typical_first_hire_day=72.0
